In [1]:
#!pip install pandas numpy 
!pip install contractions

import pandas as pd
import re
from nltk.corpus import stopwords
import contractions
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.9 MB/s eta 0:00:00


In [2]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Show loaded data
file_path = '/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv'
imdb_data = pd.read_csv(file_path)
print(imdb_data.head(10))

# Data cleaning
stop_words = set(stopwords.words('english'))

imdb_data['sentiment'] = imdb_data['sentiment'].map({'positive': 1, 'negative': 0})
imdb_data['sentiment'] = imdb_data['sentiment'].astype(int)

def clean_text(text):
    text = re.sub('<br />', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    text = contractions.fix(text)  # Expand contractions like "don't" to "do not"
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

imdb_data['review'] = imdb_data['review'].apply(clean_text)
imdb_data.head(10)

X = imdb_data['review']
y = imdb_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"Training set: {len(X_train)} reviews")
print(f"Testing set: {len(X_test)} reviews")
print(f"Training set: {len(y_train)} sentiments")
print(f"Testing set: {len(y_test)} sentiments")


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive
Training set: 45000 reviews
Testing set: 5000 reviews
Training set: 45000 sentiments
Testing set: 5000 sentiments


In [3]:
# Text Vectorization
tokenizer = Tokenizer(num_words=20000)  # Use the top 20,000 words
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
maxlen = 100  # Maximum review length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

print(f"Padded training data shape: {X_train_pad.shape}")
print(f"Padded testing data shape: {X_test_pad.shape}")

Padded training data shape: (45000, 100)
Padded testing data shape: (5000, 100)


<font size = 5 color = 'red'>GridSearchCV without KerasClassifier</font>

In [4]:
import time
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# Define the model creation function
def create_model(learning_rate=0.001, dropout_rate=0.2):
    model = Sequential([
        Embedding(input_dim=20000, output_dim=200, input_length=100),
        Dropout(dropout_rate),
        Bidirectional(LSTM(128, dropout=dropout_rate, recurrent_dropout=dropout_rate)),
        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Custom function to fit and evaluate a model
def fit_and_evaluate(params):
    model = create_model(
        learning_rate=params['learning_rate'],
        dropout_rate=params['dropout_rate']
    )
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)

    model.fit(
        X_train_pad,
        y_train,
        validation_split=0.2,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        callbacks=[early_stopping, reduce_lr],
        verbose=1  # Show progress within each model fit
    )

    # Evaluate the model
    predictions = model.predict(X_test_pad, verbose=0)
    predictions = (predictions > 0.5).astype(int).flatten()
    return accuracy_score(y_test, predictions)

# Define parameter grid
param_grid = {
    'learning_rate': [0.001, 0.0003],
    'dropout_rate': [0.15, 0.2],
    'batch_size': [32, 64],
    'epochs': [5, 10]   #Could have done 100 , but it's taking too long
}

# Perform manual grid search with progress tracking
best_params = None
best_accuracy = 0

# Total combinations
total_combinations = len(param_grid['learning_rate']) * len(param_grid['dropout_rate']) * len(param_grid['batch_size']) * len(param_grid['epochs'])
current_combination = 0

print(f"Total Parameter Combinations: {total_combinations}")
start_time = time.time()

for learning_rate in param_grid['learning_rate']:
    for dropout_rate in param_grid['dropout_rate']:
        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                current_combination += 1
                params = {
                    'learning_rate': learning_rate,
                    'dropout_rate': dropout_rate,
                    'batch_size': batch_size,
                    'epochs': epochs
                }
                print(f"\nEvaluating Combination {current_combination}/{total_combinations}: {params}")
                iteration_start = time.time()
                accuracy = fit_and_evaluate(params)
                iteration_end = time.time()

                print(f"Accuracy: {accuracy * 100:.2f}%")
                print(f"Time Taken for this Combination: {iteration_end - iteration_start:.2f} seconds")

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = params

end_time = time.time()

print(f"\nBest Parameters: {best_params}")
print(f"Best Accuracy: {best_accuracy * 100:.2f}%")
print(f"Total Time Taken: {(end_time - start_time) / 60:.2f} minutes")


Total Parameter Combinations: 16

Evaluating Combination 1/16: {'learning_rate': 0.001, 'dropout_rate': 0.15, 'batch_size': 32, 'epochs': 5}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 188s 160ms/step - accuracy: 0.7777 - loss: 0.6996 - val_accuracy: 0.8658 - val_loss: 0.3381 - learning_rate: 0.0010
Epoch 2/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.9060 - loss: 0.2553 - val_accuracy: 0.8628 - val_loss: 0.3350 - learning_rate: 0.0010
Epoch 3/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.9389 - loss: 0.1818 - val_accuracy: 0.8678 - val_loss: 0.3866 - learning_rate: 0.0010
Epoch 4/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 180s 160ms/step - accuracy: 0.9594 - loss: 0.1283 - val_accuracy: 0.8633 - val_loss: 0.4264 - learning_rate: 0.0010
Epoch 5/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.9718 - loss: 0.0905 - val_accuracy: 0.8696 - val_loss: 0.4686 - learning_rate: 2.0000e-04
Accuracy: 86.88%
Time Taken for this Combination: 914.96 seconds

Evaluating Combination 2/16: {'learning_rate': 0.001, 'dropout_rate': 0.15, 'batch_size': 32, 'epochs': 10}
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1125/1125 ━━━━━━━━━━━━━━━━━━━━ 184s 160ms/step - accuracy: 0.7717 - loss: 0.6989 - val_accuracy: 0.8744 - val_loss: 0.3126 - learning_rate: 0.0010
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 180s 160ms/step - accuracy: 0.9062 - loss: 0.2528 - val_accuracy: 0.8720 - val_loss: 0.3616 - learning_rate: 0.0010
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 180s 160ms/step - accuracy: 0.9372 - loss: 0.1775 - val_accuracy: 0.8683 - val_loss: 0.3990 - learning_rate: 0.0010
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 180s 160ms/step - accuracy: 0.9641 - loss: 0.1122 - val_accuracy: 0.8728 - val_loss: 0.4401 - learning_rate: 2.0000e-04
Accuracy: 87.58%
Time Taken for this Combination: 730.74 seconds

Evaluating Combination 3/16: {'learning_rate': 0.001, 'dropout_rate': 0.15, 'batch_size': 64, 'epochs': 5}
Epoch 1/5


...

1125/1125 ━━━━━━━━━━━━━━━━━━━━ 187s 163ms/step - accuracy: 0.7319 - loss: 1.0585 - val_accuracy: 0.8744 - val_loss: 0.3224 - learning_rate: 3.0000e-04
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 184s 164ms/step - accuracy: 0.9021 - loss: 0.2691 - val_accuracy: 0.8774 - val_loss: 0.3207 - learning_rate: 3.0000e-04
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 183s 162ms/step - accuracy: 0.9291 - loss: 0.2066 - val_accuracy: 0.8718 - val_loss: 0.3511 - learning_rate: 3.0000e-04
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 185s 164ms/step - accuracy: 0.9440 - loss: 0.1690 - val_accuracy: 0.8672 - val_loss: 0.4065 - learning_rate: 3.0000e-04
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 185s 164ms/step - accuracy: 0.9610 - loss: 0.1258 - val_accuracy: 0.8714 - val_loss: 0.4313 - learning_rate: 6.0000e-05
Accuracy: 87.66%
Time Taken for this Combination: 932.16 seconds

Evaluating Combination 15/16: {'learning_rate': 0.0003, 'dropout_rate': 0.2, 'batch_size': 64, 'epochs': 5}
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


563/563 ━━━━━━━━━━━━━━━━━━━━ 138s 239ms/step - accuracy: 0.6916 - loss: 1.3763 - val_accuracy: 0.8779 - val_loss: 0.3382 - learning_rate: 3.0000e-04
Epoch 2/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 134s 237ms/step - accuracy: 0.8970 - loss: 0.2995 - val_accuracy: 0.8720 - val_loss: 0.3166 - learning_rate: 3.0000e-04
Epoch 3/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 135s 239ms/step - accuracy: 0.9251 - loss: 0.2234 - val_accuracy: 0.8752 - val_loss: 0.3442 - learning_rate: 3.0000e-04
Epoch 4/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 135s 239ms/step - accuracy: 0.9409 - loss: 0.1827 - val_accuracy: 0.8721 - val_loss: 0.4026 - learning_rate: 3.0000e-04
Accuracy: 88.12%
Time Taken for this Combination: 549.75 seconds

Evaluating Combination 16/16: {'learning_rate': 0.0003, 'dropout_rate': 0.2, 'batch_size': 64, 'epochs': 10}
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


563/563 ━━━━━━━━━━━━━━━━━━━━ 140s 242ms/step - accuracy: 0.6996 - loss: 1.3704 - val_accuracy: 0.8698 - val_loss: 0.3599 - learning_rate: 3.0000e-04
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 136s 241ms/step - accuracy: 0.8931 - loss: 0.3058 - val_accuracy: 0.8779 - val_loss: 0.3139 - learning_rate: 3.0000e-04
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 136s 241ms/step - accuracy: 0.9218 - loss: 0.2253 - val_accuracy: 0.8766 - val_loss: 0.3536 - learning_rate: 3.0000e-04
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 137s 243ms/step - accuracy: 0.9389 - loss: 0.1877 - val_accuracy: 0.8719 - val_loss: 0.3757 - learning_rate: 3.0000e-04
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 136s 241ms/step - accuracy: 0.9569 - loss: 0.1417 - val_accuracy: 0.8714 - val_loss: 0.4106 - learning_rate: 6.0000e-05
Accuracy: 87.72%
Time Taken for this Combination: 692.25 seconds

Best Parameters: {'learning_rate': 0.0003, 'dropout_rate': 0.2, 'batch_size': 64, 'epochs': 5}
Best Accuracy: 88.12%
Total Time Taken: 201.49 min